In [1]:
from utils import create_prompt_files, run_full_analysis, to_the_forms, read_numbered_txt_files, process_all_change_requests, calculate_alignment_scores

In [3]:
import os
import pandas as pd
model_path = r"export.json"
change_requests_path = r"change_requests.txt"
api_key = os.getenv("MISTRAL_API_KEY")
agent_id = "ag:ac256ded:20250525:impact-research:a7aac9eb"

In [5]:
df= run_full_analysis(model_path, change_requests_path,1,3,5,api_key,agent_id)
to_the_forms(df, change_requests_path)


=== Running cycle 1/1 ===
Processing request 1/9...
Processing request 2/9...
Processing request 3/9...
Processing request 4/9...
Processing request 5/9...
Processing request 6/9...
Processing request 7/9...
Processing request 8/9...
Processing request 9/9...


In [4]:
create_prompt_files(change_requests_path, model_path, output_folder='to-the-chat')

Created prompt file: to-the-chat\1.txt
Created prompt file: to-the-chat\2.txt
Created prompt file: to-the-chat\3.txt
Created prompt file: to-the-chat\4.txt
Created prompt file: to-the-chat\5.txt
Created prompt file: to-the-chat\6.txt
Created prompt file: to-the-chat\7.txt
Created prompt file: to-the-chat\8.txt
Created prompt file: to-the-chat\9.txt


In [ ]:
# responses = read_numbered_txt_files("to-the-chat")
# df = process_all_change_requests(model_path, change_requests_path, 1, manual_responses=responses)

In [16]:
#df1 - LLMs response, df2 - human responce
df1 = pd.read_csv(r'outputs\to-the-calc.csv')
df2 = pd.read_csv(r'team_data\nastya.txt')

In [17]:
# Alignment for Nastya
scores, avg_score = calculate_alignment_scores(change_requests_path,df1, df2)
print(scores,"avg_score:", avg_score)

[0.16666666666666666, 0.0, 0.5, 1.0, 0.375, 0.25, 0.14285714285714285, 0.5, 0.16666666666666666] avg_score: 0.3445767195767196


In [21]:
df_n = pd.read_csv(filepath_or_buffer=r'team_data\nastya_survey.csv',sep = '\t', header = None)
df_n = df_n.drop(df_n.columns[[0,1]], axis=1).T  # Drop columns at positions 0, 1
df_n.columns = ['Relevant']
df_n.index = range(0, len(df_n))

In [33]:
df1['Relevant'] = df_n['Relevant']
relevacy = len(df1[df1["Relevant"] == "Yes"])/len(df1)
print("Relevacy:", relevacy)

Relevacy: 0.9473684210526315


In [34]:
def calculate_novelty_scores(df1, df2):
    """
    Calculate novelty scores for each change where:
    Novelty Score = (Number of new relevant components) / (Total relevant components in df1 for that change)
    
    Returns a DataFrame with Change and Novelty_Score columns
    """
    # Filter relevant components in df1 (case insensitive)
    df1_relevant = df1[df1['Relevant'].str.lower() == 'yes'].copy()
    df1_relevant['Component_lower'] = df1_relevant['Component'].str.lower()
    
    # Prepare df2 components (case insensitive)
    df2 = df2.copy()
    df2['Component_lower'] = df2['Component'].str.lower()
    
    # Get common changes between both dataframes
    common_changes = set(df1['Change']).intersection(set(df2['Change']))
    
    novelty_scores = []
    
    for change in common_changes:
        # Get components for this change in both dataframes
        df1_components = set(df1_relevant[df1_relevant['Change'] == change]['Component_lower'])
        df2_components = set(df2[df2['Change'] == change]['Component_lower'])
        
        # Calculate novelty metrics
        total_components = len(df1_components)
        new_components = len(df1_components - df2_components)
        
        if total_components > 0:  # Avoid division by zero
            novelty_score = new_components / total_components
        else:
            novelty_score = 0.0
        
        novelty_scores.append(novelty_score)
    avg_novelty = sum(novelty_scores)/len(novelty_scores) if novelty_scores else 0
    return novelty_scores, avg_novelty

# Example usage
novelty_scores, avg = calculate_novelty_scores(df1, df2)
print("Novelty", novelty_scores, "avg:", avg )

Novelty [0.5, 1.0, 0.3333333333333333, 0.0, 0.25, 0.5, 0.0, 0.5, 0.0] avg: 0.3425925925925926
